# Assignment 3 - Part 4 A #

# Validation: Numerical vs. Analytical QoI: tip deflection #

We validate the FEA result by comparing it to a numerical (by-hand) calculation using the analytical solution from Euler–Bernoulli beam theory.

---

## Problem Setup

A 2D cantilever beam is subjected to a uniform downward load along its top edge. The beam is clamped on the left (x = 0), and we compute the vertical displacement at the tip (x = L, y = H/2).

| Parameter | Description              | Value     |
|-----------|--------------------------|-----------|
| `L`       | Beam length              | 20.0      |
| `H`       | Beam height              | 1.0       |
| `q`       | Uniform load (downward)  | -0.01     |
| `E`       | Young's modulus          | 100000    |
| `ν`       | Poisson’s ratio          | 0.3       |

---

## Numerical Evaluation

The tip deflection for a cantilever under uniform load is given by:

$$ w(L) = \frac{q L^4}{8 E_{\text{eff}} I} $$

Where:

- $I = \frac{H^3}{12}$ is the second moment of area.
- $E_{\text{eff}} = \frac{E}{1 - \nu^2}$ is the effective Young’s modulus (for plane strain).

## Comparison with Code

In [ ]:
from finiteelementanalysis import pre_process as pre
from finiteelementanalysis import pre_process_demo_helper_fcns as pre_demo
from finiteelementanalysis.solver import hyperelastic_solver
from finiteelementanalysis import visualize as viz
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# for saving files later
tutorials_dir = Path.cwd()

# --- Beam geometry ---
L = 20.0   # length in x
H = 1.0    # height in y
nx = 40    # number of elements along length
ny = 2     # number of elements along height

ele_type = "D2_nn8_quad"  # 2D, 4-node quadrilateral (linear)
ndof = 2                  # 2 DOFs per node (x, y)

# Generate a rectangular mesh
coords, connect = pre.generate_rect_mesh_2d(ele_type, 0.0, 0.0, L, H, nx, ny)

# --- Identify boundaries ---
boundary_nodes, boundary_edges = pre.identify_rect_boundaries(
    coords, connect, ele_type, x_lower=0.0, x_upper=L, y_lower=0.0, y_upper=H
)

# 1) Clamp the left edge: fix x- and y-displacements = 0
fixed_left = pre.assign_fixed_nodes_rect(boundary_nodes, "left", 0.0, 0.0)

# 2) Uniform downward traction on the top edge (y=H)
q = -0.01  # load per unit length in x, downward
dload_info = pre.assign_uniform_load_rect(boundary_edges, "top", 0.0, q)

# Combine boundary conditions
fixed_nodes = fixed_left

# --- Material properties ---
E = 100000.0  # Young's modulus
nu = 0.3      # Poisson's ratio
mu = E / (2.0 * (1.0 + nu))
kappa = E / (3.0 * (1.0 - 2.0 * nu))
material_props = np.array([mu, kappa])
print(f"Material properties: mu={mu:.3f}, kappa={kappa:.3f}")

# Number of incremental load steps
nr_num_steps = 1

# --- Solve with hyperelastic solver ---
displacements_all, nr_info_all = hyperelastic_solver(
    material_props,
    ele_type,
    coords.T,
    connect.T,
    fixed_nodes,
    dload_info,
    nr_print=True,
    nr_num_steps=nr_num_steps,
    nr_tol=1e-10,
    nr_maxit=30,
)

final_disp = displacements_all[-1]

# --- Compute the tip displacement from the FEA result ---
tol = 1e-3
tip_node = None
for i, (x, y) in enumerate(coords):
    if abs(x - L) < tol and abs(y - H/2) < H/(2*ny):
        tip_node = i
        break
if tip_node is None:
    raise ValueError("Could not find tip node near x=L, y=H/2.")

tip_disp_y = final_disp[ndof*tip_node + 1]  # y-component

# --- Analytical solution using Euler-Bernoulli beam theory ---
# For plane strain, use effective Young's modulus: E_eff = E / (1 - nu^2)
E_eff = E / (1.0 - nu**2)  # Plane strain correction
I = H**3 / 12.0           # Second moment of area
w_analytical = q * L**4 / (8.0 * E_eff * I)  # Negative due to downward load

# Print results
print(f"Tip node index: {tip_node}, coordinates={coords[tip_node]}")
print(f"Computed tip deflection (y): {tip_disp_y:.6f}")
print(f"Analytical Euler-Bernoulli deflection: {w_analytical:.6f}")

# --- Evaluate error ---
error = abs(tip_disp_y - w_analytical)
print(f"Absolute error = {error:.6e}")

# --- Plot the mesh with the final deformed shape ---
img_name = "full_code_example_2.gif"
fname = str(tutorials_dir / img_name)
viz.make_deformation_gif(displacements_all, coords, connect, ele_type, fname)

# Additional verification of analytical calculation
print("\nVerification of analytical calculation:")
print(f"E_eff = {E_eff:.2f}")
print(f"I = {I:.6f}")
print(f"q * L^4 = {q * L**4:.2f}")
print(f"8 * E_eff * I = {8 * E_eff * I:.2f}")

The FEA solver computes the tip displacement using the hyperelastic formulation. After solving:

- **Analytical solution from code:** Analytical Euler-Bernoulli deflection: -0.021840
